# Pre-processing and training LDA

The purpose of this tutorial is to show you how to pre-process text data, and how to train the LDA model on that data. This tutorial will **not** explain you the LDA model, how inference is made in the LDA model, and it will not necessarily teach you how to use Gensim's implementation. There are plenty of resources for all of those things, but what is somewhat lacking is a hands-on tutorial that helps you train an LDA model with good results... so here is my contribution towards that.

I have used a corpus of NIPS papers in this tutorial, but if you're following this tutorial just to learn about LDA I encourage you to consider picking a corpus on a subject that you are familiar with. Qualitatively evaluating the output of an LDA model is challenging and can require you to understand the subject matter of your corpus (depending on your goal with the model).

I would also encourage you to consider each step when applying the model to your data, instead of just blindly applying my solution. The different steps will depend on your data and possibly your goal with the model.

In the following sections, we will go through pre-processing the data and training the model.

> **Note:**
>
> This tutorial uses the nltk library, although you can replace it with something else if you want. Python 3 is used, although Python 2.7 can be used as well.

In this tutorial we will:

* Load data.
* Pre-process data.
* Transform documents to a vectorized form.
* Train an LDA model.

If you are not familiar with the LDA model or how to use it in Gensim, I suggest you read up on that before continuing with this tutorial. Basic understanding of the LDA model should suffice. Examples:

* Gentle introduction to the LDA model: http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/
* Gensim's LDA API documentation: https://radimrehurek.com/gensim/models/ldamodel.html
* Topic modelling in Gensim: http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

## Data

We will be using some papers from the NIPS (Neural Information Processing Systems) conference. NIPS is a machine learning conference so the subject matter should be well suited for most of the target audience of this tutorial.

You can download the data from Sam Roweis' website (http://www.cs.nyu.edu/~roweis/data.html).

Note that the corpus contains 1740 documents, and not particularly long ones. So keep in mind that this tutorial is not geared towards efficiency, and be careful before applying the code to a large dataset.

Below we are simply reading the data.

In [1]:
# Read data.

import os

# Folder containing all NIPS papers.
data_dir = 'nipstxt/'

# Folders containin individual NIPS papers.
yrs = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
dirs = ['nips' + yr for yr in yrs]

# Read all texts into a list.
docs = []
for yr_dir in dirs:
    files = os.listdir(data_dir + yr_dir)
    for filen in files:
        # Note: ignoring characters that cause encoding errors.
        with open(data_dir + yr_dir + '/' + filen, errors='ignore') as fid:
            txt = fid.read()
        docs.append(txt)

## Pre-process and vectorize the documents

Among other things, we will:

* Split the documents into tokens.
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We remove numeric tokens and tokens that are only a single character, as they don't tend to be useful, and the dataset contains a lot of them.

In [2]:
# Tokenize the documents.

from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a stemmer in this case because it produces more readable words. Output that is easy to read is very desirable in topic modelling.

In [3]:
# Lemmatize the documents.

from nltk.stem.wordnet import WordNetLemmatizer

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

We find bigrams in the documents. Bigrams are sets of two adjacent words. Using bigrams we can get phrases like "machine_learning" in our output (spaces are replaced with underscores); without bigrams we would only get "machine" and "learning".

Note that in the code below, we find bigrams and then add them to the original data, because we would like to keep the words "machine" and "learning" as well as the bigram "machine_learning".

Note that computing n-grams of large dataset can be very computationally intentensive and memory intensive.

In [4]:
# Compute bigrams.

from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/Users/home/Desenvolvimento/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


We remove rare words and common words based on their *document frequency*. Below we remove words that appear in less than 20 documents or in more than 50% of the documents. Consider trying to remove words only based on their frequency, or maybe combining that with this approach.

In [5]:
# Remove rare and common tokens.

from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

#Print a map describing each word in the dictionary, and it's corresponding id in the corpus
print(dictionary.token2id)

{'solution': 0, 'assignment': 1, 'gene': 2, 'edward': 3, 'page': 4, 'department': 5, 'sc': 6, 'analog': 7, 'processor': 8, 'having': 9, 'neuron': 10, 'employed': 11, 'compute': 12, 'good': 13, 'variety': 14, 'optimization': 15, 'prob': 16, 'lem': 17, 'net': 18, 'resource': 19, 'allocation': 20, 'arises': 21, 'providing': 22, 'access': 23, 'wide': 24, 'area': 25, 'com': 26, 'energy': 27, 'mapped': 28, 'onto': 29, 'computational': 30, 'simulation': 31, 'characterizing': 32, 'communication': 33, 'field': 34, 'operation': 35, 'location': 36, 'heuristic': 37, 'finding': 38, 'feasible': 39, 'suboptimal': 40, 'developed': 41, 'previously': 42, 'recently': 43, 'hopfield': 44, 'fig': 45, 'modeled': 46, 'amplifier': 47, 'sigmoid': 48, 'curve': 49, 'synapsis': 50, 'put': 51, 'connected': 52, 'strength': 53, 'synapse': 54, 'resistive': 55, 'connection': 56, 'another': 57, 'provide': 58, 'summation': 59, 'external': 60, 'excitatory': 61, 'inhibitory': 62, 'synaptic': 63, 'link': 64, 'provides': 65,

Finally, we transform the documents to a vectorized form. We simply compute the frequency of each word, including the bigrams.

In [6]:
# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Prints the occurrence of each word in the document, in the format: (word id, how many occurrences in the document)
print(corpus[0])

[(0, 18), (1, 28), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 2), (10, 38), (11, 1), (12, 2), (13, 2), (14, 1), (15, 2), (16, 4), (17, 4), (18, 4), (19, 2), (20, 2), (21, 2), (22, 2), (23, 2), (24, 1), (25, 1), (26, 1), (27, 12), (28, 2), (29, 2), (30, 2), (31, 6), (32, 1), (33, 2), (34, 2), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 11), (45, 15), (46, 3), (47, 12), (48, 1), (49, 1), (50, 1), (51, 1), (52, 7), (53, 3), (54, 1), (55, 1), (56, 10), (57, 2), (58, 3), (59, 1), (60, 4), (61, 2), (62, 2), (63, 2), (64, 2), (65, 2), (66, 3), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 4), (75, 1), (76, 3), (77, 3), (78, 1), (79, 3), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 4), (98, 10), (99, 1), (100, 1), (101, 2), (102, 2), (103, 2), (104, 4), (105, 4), (106, 1), (107, 2), (108, 1), (109, 1), 

Let's see how many tokens and documents we have to train on.

In [7]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8640
Number of documents: 1740


## Training

We are ready to train the LDA model. We will first discuss how to set some of the training parameters.

First of all, the elephant in the room: how many topics do I need? There is really no easy answer for this, it will depend on both your data and your application. I have used 10 topics here because I wanted to have a few topics that I could interpret and "label", and because that turned out to give me reasonably good results. You might not need to interpret all your topics, so you could use a large number of topics, for example 100.

The `chunksize` controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. I've set `chunksize = 2000`, which is more than the amount of documents, so I process all the data in one go. Chunksize can however influence the quality of the model, as discussed in Hoffman and co-authors [2], but the difference was not substantial in this case.

`passes` controls how often we train the model on the entire corpus. Another word for passes might be "epochs". `iterations` is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable logging (as described in many Gensim tutorials), and set `eval_every = 1` in `LdaModel`. When training the model look for a line in the log that looks something like this:

    2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set `passes = 20` you will see this line 20 times. Make sure that by the final passes, most of the documents have converged. So you want to choose both passes and iterations to be high enough for this to happen.

We set `alpha = 'auto'` and `eta = 'auto'`. Again this is somewhat technical, but essentially we are automatically learning two parameters in the model that we usually would have to specify explicitly.

In [8]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

# Train LDA model.

from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

2017-11-22 00:51:16,586 : INFO : using autotuned alpha, starting with [0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001, 0.10000000000000001]
2017-11-22 00:51:16,591 : INFO : using serial LDA version on this node
2017-11-22 00:51:17,557 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 1740 documents, updating model once every 1740 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2017-11-22 00:51:17,560 : INFO : PROGRESS: pass 0, at document #1740/1740
2017-11-22 00:51:17,562 : DEBUG : performing inference on a chunk of 1740 documents
2017-11-22 00:52:13,921 : DEBUG : 74/1740 documents converged within 400 iterations
2017-11-22 00:52:13,965 : INFO : optimized alpha [0.059346834495654109, 0.050321668370894621, 0.099175257808575487, 0.090744046

2017-11-22 00:53:55,183 : INFO : topic #0 (0.038): 0.013*"recognition" + 0.012*"image" + 0.008*"class" + 0.007*"character" + 0.007*"classifier" + 0.006*"layer" + 0.006*"word" + 0.006*"face" + 0.004*"classification" + 0.004*"hand"
2017-11-22 00:53:55,185 : INFO : topic #8 (0.053): 0.015*"neuron" + 0.015*"cell" + 0.005*"synaptic" + 0.005*"spike" + 0.005*"response" + 0.005*"activity" + 0.005*"firing" + 0.004*"connection" + 0.004*"memory" + 0.004*"fig"
2017-11-22 00:53:55,188 : INFO : topic #3 (0.061): 0.007*"layer" + 0.006*"hidden" + 0.005*"class" + 0.005*"bound" + 0.005*"net" + 0.004*"node" + 0.004*"let" + 0.003*"theorem" + 0.003*"noise" + 0.003*"threshold"
2017-11-22 00:53:55,190 : INFO : topic #2 (0.067): 0.005*"gaussian" + 0.004*"mixture" + 0.004*"image" + 0.004*"density" + 0.003*"noise" + 0.003*"estimate" + 0.003*"class" + 0.003*"generalization" + 0.003*"prior" + 0.003*"matrix"
2017-11-22 00:53:55,192 : INFO : topic diff=0.137461, rho=0.408248
2017-11-22 00:53:55,338 : INFO : PROGRES

2017-11-22 00:55:20,405 : INFO : optimized alpha [0.035640060508461607, 0.040659974724814786, 0.06485512763979373, 0.05622874296344102, 0.040526384322693246, 0.037900429816199559, 0.028195849393114321, 0.041218163362258045, 0.044023225435116539, 0.037726618156796649]
2017-11-22 00:55:20,407 : DEBUG : updating topics
2017-11-22 00:55:20,673 : INFO : topic #6 (0.028): 0.017*"signal" + 0.011*"frequency" + 0.008*"filter" + 0.007*"sound" + 0.007*"noise" + 0.007*"auditory" + 0.006*"channel" + 0.005*"speech" + 0.005*"spike" + 0.004*"neuron"
2017-11-22 00:55:20,676 : INFO : topic #0 (0.036): 0.017*"image" + 0.015*"recognition" + 0.011*"classifier" + 0.009*"class" + 0.007*"character" + 0.006*"face" + 0.006*"classification" + 0.005*"word" + 0.005*"layer" + 0.005*"distance"
2017-11-22 00:55:20,678 : INFO : topic #8 (0.044): 0.018*"neuron" + 0.016*"cell" + 0.007*"synaptic" + 0.006*"spike" + 0.006*"activity" + 0.006*"firing" + 0.005*"response" + 0.005*"connection" + 0.004*"memory" + 0.004*"fig"
201

2017-11-22 00:56:23,219 : INFO : topic #2 (0.066): 0.006*"gaussian" + 0.005*"mixture" + 0.005*"density" + 0.004*"estimate" + 0.004*"matrix" + 0.004*"likelihood" + 0.004*"noise" + 0.004*"prior" + 0.004*"variance" + 0.004*"sample"
2017-11-22 00:56:23,221 : INFO : topic diff=0.058658, rho=0.258199
2017-11-22 00:56:23,371 : INFO : PROGRESS: pass 14, at document #1740/1740
2017-11-22 00:56:23,375 : DEBUG : performing inference on a chunk of 1740 documents
2017-11-22 00:56:37,847 : DEBUG : 1723/1740 documents converged within 400 iterations
2017-11-22 00:56:37,888 : INFO : optimized alpha [0.035882548315090551, 0.044084148745178853, 0.066551292535415377, 0.055196924689226629, 0.039692996485623716, 0.035631302740588004, 0.026726352186176354, 0.042472754908465166, 0.041337467549034911, 0.035058685024143131]
2017-11-22 00:56:37,890 : DEBUG : updating topics
2017-11-22 00:56:38,131 : INFO : topic #6 (0.027): 0.021*"signal" + 0.012*"frequency" + 0.010*"filter" + 0.009*"noise" + 0.008*"sound" + 0.

2017-11-22 00:57:35,392 : INFO : topic #1 (0.047): 0.014*"hidden" + 0.009*"layer" + 0.009*"net" + 0.008*"hidden_unit" + 0.007*"speech" + 0.006*"trained" + 0.006*"word" + 0.006*"architecture" + 0.005*"rule" + 0.005*"recognition"
2017-11-22 00:57:35,394 : INFO : topic #3 (0.055): 0.008*"bound" + 0.007*"let" + 0.006*"theorem" + 0.006*"class" + 0.006*"layer" + 0.005*"node" + 0.005*"threshold" + 0.004*"net" + 0.004*"hidden" + 0.004*"approximation"
2017-11-22 00:57:35,396 : INFO : topic #2 (0.068): 0.006*"gaussian" + 0.005*"mixture" + 0.005*"density" + 0.004*"estimate" + 0.004*"matrix" + 0.004*"likelihood" + 0.004*"noise" + 0.004*"prior" + 0.004*"variance" + 0.004*"component"
2017-11-22 00:57:35,399 : INFO : topic diff=0.049759, rho=0.223607
2017-11-22 00:57:35,560 : INFO : PROGRESS: pass 19, at document #1740/1740
2017-11-22 00:57:35,562 : DEBUG : performing inference on a chunk of 1740 documents
2017-11-22 00:57:48,427 : DEBUG : 1727/1740 documents converged within 400 iterations
2017-11-2

CPU times: user 6min 25s, sys: 3.52 s, total: 6min 28s
Wall time: 6min 32s


We can compute the topic coherence of each topic. Below we display the average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see docs, https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel.top_topics), Gensim has recently obtained an implementation of the "AKSW" topic coherence measure (see accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can see that the topics below make a lot of sense. However, they are not without flaws. We can see that there is substantial overlap between some topics, others are hard to interpret, and most of them have at least some terms that seem out of place. If you were able to do better, feel free to share your methods on the blog at http://rare-technologies.com/lda-training-tips/ !

In [9]:
top_topics = model.top_topics(corpus, topn=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2017-11-22 00:57:48,886 : DEBUG : Setting topics to those of the model: LdaModel(num_terms=8640, num_topics=10, decay=0.5, chunksize=2000)
2017-11-22 00:57:49,033 : INFO : CorpusAccumulator accumulated stats from 1000 documents


Average topic coherence: -1.2563.
[([(0.02082871225888288, 'neuron'),
   (0.017912085524094061, 'cell'),
   (0.0074868226254679896, 'synaptic'),
   (0.0071335218934656733, 'activity'),
   (0.0069042615445139115, 'spike'),
   (0.0064441650840668151, 'firing'),
   (0.0056003169669445593, 'response'),
   (0.0054292117946804982, 'connection'),
   (0.0044503452629721446, 'cortex'),
   (0.0044308388169116454, 'stimulus'),
   (0.0041241319350052603, 'cortical'),
   (0.0041205459638535506, 'potential'),
   (0.003974626971486284, 'fig'),
   (0.003698179590656824, 'simulation'),
   (0.0035269393380769139, 'synapsis'),
   (0.0034240657970605439, 'memory'),
   (0.0032928184556285009, 'excitatory'),
   (0.0032334982039190093, 'inhibitory'),
   (0.0031353996560506353, 'phase'),
   (0.0030796398318785672, 'layer')],
  -0.93310519368200007),
 ([(0.014143832823105511, 'hidden'),
   (0.0089530381405426469, 'layer'),
   (0.0085584006535876327, 'net'),
   (0.0078528002284972351, 'hidden_unit'),
   (0.0070

### Saving the models for later...

In [10]:
dictionary.save('dicionario_1')
bigram.save('frases_1')
model.save('modelo_1')

2017-11-22 00:57:49,286 : INFO : saving Dictionary object under dicionario_1, separately None
2017-11-22 00:57:49,298 : INFO : saved dicionario_1
2017-11-22 00:57:49,302 : INFO : saving Phrases object under frases_1, separately None
2017-11-22 00:57:51,413 : INFO : saved frases_1
2017-11-22 00:57:51,416 : INFO : saving LdaState object under modelo_1.state, separately None
2017-11-22 00:57:51,427 : INFO : saved modelo_1.state
2017-11-22 00:57:51,432 : INFO : saving LdaModel object under modelo_1, separately ['expElogbeta', 'sstats']
2017-11-22 00:57:51,433 : INFO : storing np array 'expElogbeta' to modelo_1.expElogbeta.npy
2017-11-22 00:57:51,438 : INFO : not storing attribute state
2017-11-22 00:57:51,440 : INFO : not storing attribute id2word
2017-11-22 00:57:51,441 : INFO : not storing attribute dispatcher
2017-11-22 00:57:51,444 : INFO : saved modelo_1


## Memory Considerations

Gensim can only do so much to limit the amount of memory used by your analysis. Your program may take an extended amount of time or possibly crash if you do not take into account the amount of memory the program will consume. Prior to training your model you can get a ballpark estimate of memory use by using the following formula:

### 8 bytes * num_terms * num_topics * 3

#### 8 bytes
size of double precision float
#### num_terms 
number of terms in the dictionary
#### num_topics 
number of topics
#### The magic number 3 
The 8 bytes * num_terms * num_topic accounts for the model output, but Gensim will need to make temporary copies while modeling. The scaling factor of 3 gives you an idea of how much memory Gensim will be consuming while running with the temporary copies present.

## Things to experiment with

* `no_above` and `no_below` parameters in `filter_extremes` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

If you have other ideas, feel free to comment on http://rare-technologies.com/lda-training-tips/.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.